<a href="https://colab.research.google.com/github/guitorte/audio/blob/main/LLM_Lyrics_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Lyrics Generator

Generate song lyrics using pre-trained Large Language Models — **no training required**.

This notebook provides two backends:

| Backend | Quality | API Key? | Cost | Best For |
|---------|---------|----------|------|----------|
| **Google Gemini** | Excellent | Yes (free tier) | Free | Production-quality lyrics |
| **Hugging Face (local)** | Good | No | Free | Quick experiments, offline use |

## How it works

1. Choose your LLM backend
2. Set parameters: genre, mood, theme, language, song structure
3. Generate complete lyrics with verses, chorus, bridge, etc.
4. Iterate and refine
5. Export to Google Drive

In [ ]:
# @title 1. Install Dependencies
# Installs all required packages for both Gemini and Hugging Face backends.

print("Installing dependencies...")
!pip install -q google-generativeai transformers accelerate sentencepiece protobuf
print("All dependencies installed.")

In [ ]:
# @title 2. Mount Google Drive (Optional)
# Mount your Google Drive to save generated lyrics.
# Skip this step if you only want to view lyrics in the notebook.

mount_drive = True  # @param {type:"boolean"}

if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_DIR = "/content/drive/MyDrive/generated_lyrics"
else:
    SAVE_DIR = "/content/generated_lyrics"

import os
os.makedirs(SAVE_DIR, exist_ok=True)
print(f"Lyrics will be saved to: {SAVE_DIR}")

In [ ]:
# @title 3. Choose Your LLM Backend
# Select which model to use for lyrics generation.
#
# - "gemini": Uses Google's Gemini API (free tier: 15 requests/minute).
#   Get your free API key at https://aistudio.google.com/apikey
#
# - "huggingface": Downloads and runs a model locally on the Colab GPU.
#   No API key needed. Uses more RAM but works fully offline after download.

backend = "gemini"  # @param ["gemini", "huggingface"]

# --- Gemini Configuration ---
# Paste your free Gemini API key here (from https://aistudio.google.com/apikey)
gemini_api_key = ""  # @param {type:"string"}

# --- Hugging Face Configuration ---
# Model to use. Recommended options:
#   - "mistralai/Mistral-7B-Instruct-v0.3" (best quality, needs ~16GB GPU RAM)
#   - "google/gemma-2b-it" (good balance, works on T4 GPU)
#   - "TinyLlama/TinyLlama-1.1B-Chat-v1.0" (lightweight, works on any GPU)
hf_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # @param {type:"string"}

# --- Initialization ---
model_instance = None
tokenizer_instance = None

if backend == "gemini":
    if not gemini_api_key:
        print("ERROR: Gemini API key is required.")
        print("Get a free key at: https://aistudio.google.com/apikey")
        print("Paste it in the 'gemini_api_key' field above and re-run this cell.")
    else:
        import google.generativeai as genai
        genai.configure(api_key=gemini_api_key)
        model_instance = genai.GenerativeModel("gemini-2.0-flash")
        print(f"Gemini backend ready (model: gemini-2.0-flash).")
        print("Free tier: 15 requests/minute, 1500 requests/day.")

elif backend == "huggingface":
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Device: {device}")
    if device == "cpu":
        print("WARNING: No GPU detected. Generation will be slow.")
        print("Go to Runtime > Change runtime type > Select T4 GPU.")

    print(f"Loading model: {hf_model_name}")
    print("This only happens once per session (model is cached in memory).")

    tokenizer_instance = AutoTokenizer.from_pretrained(hf_model_name)
    model_instance = AutoModelForCausalLM.from_pretrained(
        hf_model_name,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        device_map="auto",
    )
    print(f"Model loaded and ready on {device}.")

In [ ]:
# @title 4. Lyrics Generation Engine (Run Once)
# This cell defines the core generation functions. Run it once to load them.

import re
import os
from datetime import datetime


def build_lyrics_prompt(genre, mood, theme, language, structure, extra_instructions,
                        title_suggestion, reference_artist):
    """Build a detailed prompt for lyrics generation."""
    parts = []
    parts.append("You are a professional songwriter and lyricist.")
    parts.append(f"Write complete song lyrics in {language}.")
    parts.append("")

    parts.append("SONG SPECIFICATIONS:")
    parts.append(f"- Genre: {genre}")
    parts.append(f"- Mood/Feeling: {mood}")
    parts.append(f"- Theme/Subject: {theme}")

    if title_suggestion:
        parts.append(f"- Suggested Title: {title_suggestion}")
    if reference_artist:
        parts.append(f"- Style Reference: Write in a style inspired by {reference_artist}")

    parts.append("")
    parts.append("SONG STRUCTURE:")
    for i, section in enumerate(structure, 1):
        parts.append(f"  {i}. {section}")

    parts.append("")
    parts.append("FORMATTING RULES:")
    parts.append("- Label each section clearly in square brackets, e.g. [Verse 1], [Chorus], [Bridge]")
    parts.append("- Leave a blank line between sections.")
    parts.append("- Write ONLY the lyrics with section labels. No explanations or commentary.")
    parts.append(f"- The lyrics MUST be entirely in {language}.")
    parts.append("- Use natural phrasing and rhythm appropriate for singing.")
    parts.append("- Make the chorus memorable and catchy.")

    if extra_instructions:
        parts.append("")
        parts.append(f"ADDITIONAL INSTRUCTIONS: {extra_instructions}")

    return "\n".join(parts)


def generate_with_gemini(prompt, temperature=0.9, max_tokens=2048):
    """Generate lyrics using the Gemini API."""
    generation_config = {
        "temperature": temperature,
        "max_output_tokens": max_tokens,
        "top_p": 0.95,
        "top_k": 40,
    }
    response = model_instance.generate_content(
        prompt,
        generation_config=generation_config,
    )
    return response.text


def generate_with_huggingface(prompt, temperature=0.9, max_tokens=1024):
    """Generate lyrics using a local Hugging Face model."""
    import torch

    # Format as chat for instruction-tuned models
    if hasattr(tokenizer_instance, 'apply_chat_template'):
        messages = [{"role": "user", "content": prompt}]
        formatted = tokenizer_instance.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
    else:
        formatted = prompt

    inputs = tokenizer_instance(formatted, return_tensors="pt").to(model_instance.device)
    input_length = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model_instance.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            top_k=50,
            do_sample=True,
            repetition_penalty=1.2,
            pad_token_id=tokenizer_instance.eos_token_id,
        )

    generated_tokens = outputs[0][input_length:]
    return tokenizer_instance.decode(generated_tokens, skip_special_tokens=True)


def generate_lyrics(genre, mood, theme, language="English",
                    structure=None, extra_instructions="",
                    title_suggestion="", reference_artist="",
                    temperature=0.9, max_tokens=2048):
    """Generate song lyrics with the configured backend."""

    if structure is None:
        structure = ["Verse 1", "Chorus", "Verse 2", "Chorus", "Bridge", "Chorus"]

    prompt = build_lyrics_prompt(
        genre, mood, theme, language, structure,
        extra_instructions, title_suggestion, reference_artist
    )

    print(f"Generating {genre} lyrics about '{theme}'...")
    print(f"Backend: {backend} | Temperature: {temperature}")
    print("-" * 50)

    if backend == "gemini":
        lyrics = generate_with_gemini(prompt, temperature, max_tokens)
    elif backend == "huggingface":
        lyrics = generate_with_huggingface(prompt, temperature, min(max_tokens, 1024))
    else:
        raise ValueError(f"Unknown backend: {backend}")

    return lyrics.strip()


def save_lyrics(lyrics, title, genre, save_dir=None):
    """Save lyrics to a text file."""
    if save_dir is None:
        save_dir = SAVE_DIR

    safe_title = re.sub(r'[^\w\s-]', '', title).strip().replace(' ', '_')
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{safe_title}_{genre}_{timestamp}.txt"
    filepath = os.path.join(save_dir, filename)

    header = f"Title: {title}\nGenre: {genre}\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M')}\nBackend: {backend}\n"
    header += "=" * 50 + "\n\n"

    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(header + lyrics)

    print(f"\nSaved to: {filepath}")
    return filepath


def refine_lyrics(original_lyrics, feedback, temperature=0.7):
    """Refine previously generated lyrics based on feedback."""
    prompt = (
        "You are a professional songwriter. Here are song lyrics that need revision:\n\n"
        f"{original_lyrics}\n\n"
        f"REVISION REQUEST: {feedback}\n\n"
        "Rewrite the complete lyrics incorporating the requested changes. "
        "Keep the same section labels and overall structure unless asked to change them. "
        "Output ONLY the revised lyrics with section labels, no explanations."
    )

    print(f"Refining lyrics based on: '{feedback}'...")
    print("-" * 50)

    if backend == "gemini":
        return generate_with_gemini(prompt, temperature).strip()
    elif backend == "huggingface":
        return generate_with_huggingface(prompt, temperature).strip()


print("Lyrics generation engine loaded.")
print("Available functions:")
print("  generate_lyrics()  - Generate new lyrics")
print("  refine_lyrics()    - Refine existing lyrics")
print("  save_lyrics()      - Save lyrics to file")

In [ ]:
# @title 5. Generate Lyrics
# Configure your song parameters and generate lyrics.

# --- Song Parameters ---
genre = "Pop"  # @param ["Pop", "Rock", "Hip-Hop/Rap", "R&B/Soul", "Country", "Jazz", "Blues", "Reggae", "Electronic/EDM", "Folk", "Metal", "Punk", "Indie", "Latin", "Samba", "Pagode", "Sertanejo", "Bossa Nova", "MPB", "Funk Carioca", "Forro", "Gospel", "K-Pop", "Afrobeats", "Classical Crossover"]
mood = "Upbeat and hopeful"  # @param ["Upbeat and hopeful", "Melancholic and reflective", "Angry and rebellious", "Romantic and passionate", "Nostalgic and wistful", "Empowering and confident", "Dark and mysterious", "Peaceful and calming", "Energetic and wild", "Bittersweet", "Playful and fun", "Spiritual and transcendent"]
theme = "Finding your own path in life"  # @param {type:"string"}
language = "English"  # @param ["English", "Portuguese", "Spanish", "French", "Italian", "German", "Japanese", "Korean"]

# --- Optional Parameters ---
title_suggestion = ""  # @param {type:"string"}
reference_artist = ""  # @param {type:"string"}
extra_instructions = ""  # @param {type:"string"}

# --- Song Structure ---
# Customize the structure or leave as default
structure_preset = "Standard (Verse-Chorus-Verse-Chorus-Bridge-Chorus)"  # @param ["Standard (Verse-Chorus-Verse-Chorus-Bridge-Chorus)", "Extended (Intro-Verse-PreChorus-Chorus-Verse-PreChorus-Chorus-Bridge-Chorus-Outro)", "Simple (Verse-Chorus-Verse-Chorus)", "Rap (Intro-Verse1-Hook-Verse2-Hook-Verse3-Hook-Outro)", "Ballad (Intro-Verse1-Verse2-Chorus-Verse3-Chorus-Bridge-Chorus)", "Custom"]

# Only used if structure_preset is "Custom"
custom_structure = "Verse 1, Chorus, Verse 2, Chorus, Bridge, Chorus"  # @param {type:"string"}

# --- Generation Settings ---
temperature = 0.9  # @param {type:"slider", min:0.1, max:1.5, step:0.1}
max_tokens = 2048  # @param {type:"slider", min:512, max:4096, step:256}

# --- Build Structure ---
STRUCTURE_MAP = {
    "Standard (Verse-Chorus-Verse-Chorus-Bridge-Chorus)": [
        "Verse 1", "Chorus", "Verse 2", "Chorus", "Bridge", "Chorus"
    ],
    "Extended (Intro-Verse-PreChorus-Chorus-Verse-PreChorus-Chorus-Bridge-Chorus-Outro)": [
        "Intro", "Verse 1", "Pre-Chorus", "Chorus", "Verse 2", "Pre-Chorus",
        "Chorus", "Bridge", "Chorus", "Outro"
    ],
    "Simple (Verse-Chorus-Verse-Chorus)": [
        "Verse 1", "Chorus", "Verse 2", "Chorus"
    ],
    "Rap (Intro-Verse1-Hook-Verse2-Hook-Verse3-Hook-Outro)": [
        "Intro", "Verse 1", "Hook", "Verse 2", "Hook", "Verse 3", "Hook", "Outro"
    ],
    "Ballad (Intro-Verse1-Verse2-Chorus-Verse3-Chorus-Bridge-Chorus)": [
        "Intro", "Verse 1", "Verse 2", "Chorus", "Verse 3", "Chorus",
        "Bridge", "Chorus"
    ],
}

if structure_preset == "Custom":
    structure = [s.strip() for s in custom_structure.split(",")]
else:
    structure = STRUCTURE_MAP[structure_preset]

# --- Generate ---
lyrics = generate_lyrics(
    genre=genre,
    mood=mood,
    theme=theme,
    language=language,
    structure=structure,
    extra_instructions=extra_instructions,
    title_suggestion=title_suggestion,
    reference_artist=reference_artist,
    temperature=temperature,
    max_tokens=max_tokens,
)

print("\n" + "=" * 50)
print("GENERATED LYRICS")
print("=" * 50 + "\n")
print(lyrics)

In [ ]:
# @title 6. Refine Lyrics (Optional)
# Provide feedback to revise the lyrics generated in the previous step.
# Re-run this cell multiple times to iterate.

feedback = "Make the chorus more repetitive and catchy"  # @param {type:"string"}
refinement_temperature = 0.7  # @param {type:"slider", min:0.1, max:1.5, step:0.1}

if not feedback:
    print("Enter your feedback in the 'feedback' field above and re-run.")
else:
    lyrics = refine_lyrics(lyrics, feedback, temperature=refinement_temperature)
    print("\n" + "=" * 50)
    print("REFINED LYRICS")
    print("=" * 50 + "\n")
    print(lyrics)

In [ ]:
# @title 7. Save Lyrics to File
# Save the current lyrics (from generation or refinement) to a text file.

song_title = "My Song"  # @param {type:"string"}

if 'lyrics' not in dir() or not lyrics:
    print("No lyrics to save. Run the generation step first (Step 5).")
else:
    save_lyrics(lyrics, song_title, genre)

In [ ]:
# @title 8. Batch Generation - Multiple Variations
# Generate multiple variations of the same song concept.
# Useful for picking the best version or combining elements from different takes.

num_variations = 3  # @param {type:"slider", min:2, max:5, step:1}
batch_genre = "Pop"  # @param ["Pop", "Rock", "Hip-Hop/Rap", "R&B/Soul", "Country", "Jazz", "Blues", "Reggae", "Electronic/EDM", "Folk", "Metal", "Punk", "Indie", "Latin", "Samba", "Pagode", "Sertanejo", "Bossa Nova", "MPB", "Funk Carioca", "Forro", "Gospel", "K-Pop", "Afrobeats", "Classical Crossover"]
batch_mood = "Romantic and passionate"  # @param ["Upbeat and hopeful", "Melancholic and reflective", "Angry and rebellious", "Romantic and passionate", "Nostalgic and wistful", "Empowering and confident", "Dark and mysterious", "Peaceful and calming", "Energetic and wild", "Bittersweet", "Playful and fun", "Spiritual and transcendent"]
batch_theme = "Summer love"  # @param {type:"string"}
batch_language = "English"  # @param ["English", "Portuguese", "Spanish", "French", "Italian", "German", "Japanese", "Korean"]
batch_title = "Summer Love"  # @param {type:"string"}

import time

all_variations = []

for i in range(num_variations):
    print(f"\n{'#' * 60}")
    print(f"  VARIATION {i + 1} of {num_variations}")
    print(f"{'#' * 60}\n")

    # Vary temperature slightly for each take
    temp = 0.8 + (i * 0.15)

    variation = generate_lyrics(
        genre=batch_genre,
        mood=batch_mood,
        theme=batch_theme,
        language=batch_language,
        title_suggestion=batch_title,
        temperature=temp,
    )

    all_variations.append(variation)
    print("\n" + variation)

    # Save each variation
    save_lyrics(variation, f"{batch_title}_v{i+1}", batch_genre)

    # Small delay to avoid rate limiting with Gemini
    if backend == "gemini" and i < num_variations - 1:
        time.sleep(2)

print(f"\n{'=' * 60}")
print(f"Generated {num_variations} variations. All saved to: {SAVE_DIR}")
print(f"{'=' * 60}")

In [ ]:
# @title 9. Genre Adapter - Convert Lyrics to a Different Genre
# Take existing lyrics and adapt them to a completely different genre and style.

target_genre = "Bossa Nova"  # @param ["Pop", "Rock", "Hip-Hop/Rap", "R&B/Soul", "Country", "Jazz", "Blues", "Reggae", "Electronic/EDM", "Folk", "Metal", "Punk", "Indie", "Latin", "Samba", "Pagode", "Sertanejo", "Bossa Nova", "MPB", "Funk Carioca", "Forro", "Gospel", "K-Pop", "Afrobeats"]
target_language = "Portuguese"  # @param ["Keep original", "English", "Portuguese", "Spanish", "French", "Italian", "German", "Japanese", "Korean"]
adaptation_temp = 0.85  # @param {type:"slider", min:0.1, max:1.5, step:0.05}

if 'lyrics' not in dir() or not lyrics:
    print("No lyrics to adapt. Run the generation step first (Step 5).")
else:
    lang_instruction = ""
    if target_language != "Keep original":
        lang_instruction = f" Translate the lyrics to {target_language} while adapting them."

    adapt_prompt = (
        f"You are a professional songwriter and music arranger.\n\n"
        f"Here are the original lyrics:\n\n{lyrics}\n\n"
        f"Adapt these lyrics for the {target_genre} genre. "
        f"Change the vocabulary, rhythm, phrasing, and imagery to fit {target_genre} conventions. "
        f"Keep the core theme and emotional message but make it feel authentic to {target_genre}."
        f"{lang_instruction}\n\n"
        f"Output ONLY the adapted lyrics with section labels in square brackets. No explanations."
    )

    print(f"Adapting lyrics to {target_genre}...")
    if target_language != "Keep original":
        print(f"Translating to {target_language}...")
    print("-" * 50)

    if backend == "gemini":
        adapted_lyrics = generate_with_gemini(adapt_prompt, adaptation_temp).strip()
    else:
        adapted_lyrics = generate_with_huggingface(adapt_prompt, adaptation_temp).strip()

    print("\n" + "=" * 50)
    print(f"ADAPTED LYRICS ({target_genre})")
    print("=" * 50 + "\n")
    print(adapted_lyrics)

    # Offer to save
    adapted_title = f"Adapted_{target_genre.replace('/', '-')}"
    save_lyrics(adapted_lyrics, adapted_title, target_genre)

---

## Tips for Better Results

### Temperature Guide
| Value | Effect |
|-------|--------|
| 0.3-0.5 | Conservative, predictable phrasing |
| 0.7-0.9 | Balanced creativity (recommended) |
| 1.0-1.3 | More experimental, unexpected word choices |
| 1.3+ | Very creative but may lose coherence |

### Extra Instructions Examples
- `"Use metaphors about the ocean and sailing"`
- `"Include a call-and-response pattern in the chorus"`
- `"Make verse 2 tell the story from a different perspective"`
- `"Use internal rhyme schemes, not just end rhymes"`
- `"Write in first person plural (we/us)"`

### Workflow
1. Start with **Step 5** to generate initial lyrics
2. Use **Step 6** to iterate and refine (run multiple times)
3. Try **Step 8** to generate variations and pick the best one
4. Use **Step 9** to adapt into different genres/languages
5. Save your favorites with **Step 7**